In [1]:
import pandas as pd 
import numpy as np

# DT 문제 1번

#### ※함수에 들어가 있는 변수나 flow는 본인이 바꾸셔도 가능하며 결과만 똑같이 나오면 됩니다!
#### ※hard코딩(이 데이터셋에만 적용되는 코딩방법) 말고 전체 데이터에 적용 가능하게 함수를 짜주셔야 합니다.

# Data Loading

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수에 대한 함수 만들기.

- Input은 Dataframe과 label이름으로 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다!

In [203]:
def get_gini(df,label):
    
    # label에 따른 분포 정보
    pair = df[label].value_counts()
    #print('pair: ',pair)
    feature_value = pair.values
    #print('value: ', feature_value)
    # 분모
    under = feature_value.sum()
    #print(under)
    
    
    gini = 1
    for i in feature_value:
        gini -=(i/under)*(i/under) 
    return gini

In [4]:
get_gini(pd_data, 'class_buys_computer')

pair:  yes    9
no     5
Name: class_buys_computer, dtype: int64
value:  [9 5]
14


0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})
- Powerset은 모든 조합을 뱉어내게 됩니다. 이건 그냥 완성된걸 드릴게요.

In [5]:
from itertools import chain, combinations

def powerset(feature_class):
    listed_data = list(feature_class)
    chain_set = chain.from_iterable(combinations(listed_data, i) 
                                    for i in range(len(listed_data)+1))
    return [set_data for set_data in chain_set]

In [6]:
powerset(pd_data.age.unique())

[(),
 ('youth',),
 ('middle_aged',),
 ('senior',),
 ('youth', 'middle_aged'),
 ('youth', 'senior'),
 ('middle_aged', 'senior'),
 ('youth', 'middle_aged', 'senior')]

- 저 위에 부분 집합 중 우리가 원하는 집합(이진 분류)만 골라 내야하겠죠?
- 그 함수를 get_binary_split로 완성해주세요!
- 완성된 내용은 다음과 같이 나와야 합니다.

In [141]:
def get_binary_split(df, attribute):
    feature_category = df[attribute].unique()
    #print(feature_category)
    
    result = [i for i in powerset(df[attribute].unique()) if len(i)>0 and len(i)<len(df[attribute].unique())]
    result = [list(i) for i in result]
#    print(result)
    empty = []


    for idx, val in enumerate(result):
        current_val = val
        else_val =  result[:idx] + result[idx+1 :]
        #print('val: ',val)
        #print('else: ',else_val)
        for idx_2, val_2 in enumerate(else_val):
            current_combi = [current_val,val_2]
            #print('current_combi',current_combi)
            
            flat_current_combi = sum(current_combi, [])
            #print('flat_current_combi',flat_current_combi)
            check_len = len(flat_current_combi)
            #print('flat_current_combi',flat_current_combi)
            if set(flat_current_combi) == set(feature_category) and check_len == len(feature_category): 
                #print('happy')
                #print(current_combi)
                empty.append(current_combi)
    
    return empty

In [142]:
get_binary_split(pd_data, "age")

[[['youth'], ['middle_aged', 'senior']],
 [['middle_aged'], ['youth', 'senior']],
 [['senior'], ['youth', 'middle_aged']],
 [['youth', 'middle_aged'], ['senior']],
 [['youth', 'senior'], ['middle_aged']],
 [['middle_aged', 'senior'], ['youth']]]

In [200]:

binary= get_binary_split(pd_data, "age")

for check in binary:
    print(check[0],check[1])
    A=pd_data.loc[pd_data['age'].isin(check[0])]
    B=pd_data.loc[pd_data['age'].isin(check[1])]
    print(A.shape[0])
    print(B.shape[0])



['youth'] ['middle_aged', 'senior']
5
9
['middle_aged'] ['youth', 'senior']
4
10
['senior'] ['youth', 'middle_aged']
5
9
['youth', 'middle_aged'] ['senior']
9
5
['youth', 'senior'] ['middle_aged']
10
4
['middle_aged', 'senior'] ['youth']
9
5


# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 함수를 사용하여 만들어주세요!
- DataFrame의 index를 사용하여 만들면 굉장히 편합니다..! 예시를 아래에서 보여드릴게요.
- 결과는 아래 아래 아래 줄과 같아야 합니다.

In [274]:
def get_attribute_gini_index(df, attribute, label):
    binary = get_binary_split(df,attribute)
    sub_gini = dict()
    
    for check in binary:
        gini_left_data = df.loc[df[attribute].isin(check[0])]
        gini_right_data = df.loc[df[attribute].isin(check[1])]
        
        gini_left = get_gini(gini_left_data,label)
        gini_right = get_gini(gini_right_data,label)
        
        count_left = gini_left_data.shape[0]
        count_right = gini_right_data.shape[0]
        under = count_left + count_right
        
        gini = ((count_left/under) * gini_left) + ((count_right/under) * gini_right)
        
        key_dict = ''
        #print(check[0])
        if len(check[0]) == 1:
            key_dict = (check[0][0])
            #print(key_dict)
        else:
            #print(check[0])
            key_dict = ' & '.join(str(e) for e in check[0])

        #print(check[1][:])
        
        sub_gini[key_dict] = gini
    return sub_gini
        
        
        

In [275]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'middle_aged': 0.35714285714285715,
 'middle_aged & senior': 0.3936507936507937,
 'senior': 0.4571428571428572,
 'youth': 0.3936507936507937,
 'youth & middle_aged': 0.4571428571428572,
 'youth & senior': 0.35714285714285715}

- 여기서 가장 작은값으로 분류를 해야겠죠?

In [277]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) income 의 이진분류를 얻는 함수 get_binary_split(pd_data, "income")을 통해 보여주세요.

In [278]:
get_binary_split(pd_data,'income')

[[['high'], ['medium', 'low']],
 [['medium'], ['high', 'low']],
 [['low'], ['high', 'medium']],
 [['high', 'medium'], ['low']],
 [['high', 'low'], ['medium']],
 [['medium', 'low'], ['high']]]

## 문제2) 가장 Gini계수가 높은 Feature 즉 분류를 하는데 가장 중요한 변수를 선정하시고 get_attribute_gini_index함수를 통해 Gini index를 제시해주세요.

In [279]:
for attr in pd_data.columns[:-1]:
    print(attr,' : ',min(get_attribute_gini_index(pd_data, attr, "class_buys_computer").items()))
    
'''age의 gini index = 0.3571로 가장 작다'''    

age  :  ('middle_aged', 0.35714285714285715)
income  :  ('high', 0.4428571428571429)
student  :  ('no', 0.3673469387755103)
credit_rating  :  ('excellent', 0.42857142857142855)


'age의 gini index = 0.3571로 가장 작다'

## 문제3) 2에서 구한 Feature로 DataFrame을 분류 해주시고 나눠진 2개의 클래스에서 각각 다음으로 중요한 Feature를 선정해주시고 Gini index를 제시해주세요.

In [280]:
'''먼저 age - middle vs youth,senior'''
data_2_left=pd_data[pd_data['age']=='middle_aged']
data_2_right=pd_data[pd_data['age']!='middle_aged']

In [281]:
data_2_left

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


In [282]:
data_2_right

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes
10,youth,medium,yes,excellent,yes
13,senior,medium,no,excellent,no


In [283]:
'''그 다음 split point가 되는 피쳐를 찾자 '''
for attr in data_2_right.columns[:-1]:
    print(attr,' : ',min(get_attribute_gini_index(data_2_right, attr, "class_buys_computer").items()))
    
'''student의 gini index = 0.3199로 가장 작다'''

age  :  ('senior', 0.48)
income  :  ('high', 0.375)
student  :  ('no', 0.31999999999999984)
credit_rating  :  ('excellent', 0.4166666666666667)


'student의 gini index = 0.3199로 가장 작다'